### Install dependencies

In [1]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U

### Load CSV data from S3

In [2]:
!pwd

/home/reson/Documents/GitHub/Enterprise-RAG/notebooks


In [1]:
import boto3
import pandas as pd

s3 = boto3.client('s3')

bucket_name = 'recipes-rag'
file_key = 'food_recipes.csv'

# Download the file from s3 locally
s3.download_file(bucket_name, file_key, '../data/food_recipes.csv')

# Load the CSV into a DataFrame
df = pd.read_csv('../data/food_recipes.csv')

df.head()

,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,good,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15 M,15 M,Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...,To begin making the Roasted Peppers And Mushro...,Divya Shivaraman,Party Food Recipes|Tea Party Recipes|Mushroom ...,Pizza Recipes
1,Thakkali Gotsu Recipe | Thakkali Curry | Spicy...,https://www.archanaskitchen.com/tomato-gotsu-r...,good,3423,4.932223,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,10 M,20 M,Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Archana Doshi,Vegetarian Recipes|Tomato Recipes|South Indian...,Indian Curry Recipes
2,Spicy Grilled Pineapple Salsa Recipe,https://www.archanaskitchen.com/spicy-grilled-...,good,2091,4.945959,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,10 M,0 M,Extra Virgin Olive Oil|Pineapple|White onion|R...,To begin making the Spicy Grilled Pineapple Sa...,Archana's Kitchen,Party Starter & Appetizer Recipes|Pineapple Re...,Mexican Recipes
3,Karwar Style Dali Thoy Recipe - Toor dal Curry,https://www.archanaskitchen.com/dali-thoy-reci...,good,990,4.888889,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,5 M,20 M,Arhar dal (Split Toor Dal)|Turmeric powder (Ha...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Jyothi Rajesh,Side Dish Recipes|South Indian Recipes|Indian ...,Indian Curry Recipes
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,https://www.archanaskitchen.com/rajma-kofta-in...,good,345,4.828986,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,20 M,30 M,Rajma (Large Kidney Beans)|Cashew nuts|Sultana...,To begin making Rajma Kofta In Milk And Poppy ...,RUBY PATHAK,Side Dish Recipes|Indian Lunch Recipes|Office ...,Kofta Recipes


### Load data into chunked documents

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [3]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/home/reson/miniconda3/envs/LangChain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/reson/miniconda3/envs/LangChain/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
text_splitter_recursive = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)

In [8]:
def create_documents(df):
    # AWS Kendra requires non empty string values for each field
    # Make explicit to the LLM that the field is not available
    df.fillna('Not Available', inplace=True)
    
    documents = []
    for index, row in df.iterrows():
        metadata = {
            'recipe_title': str(row['recipe_title']) if row['recipe_title'] else 'No Title Available',
            'url': str(row['url']) if row['url'] else 'https://example.com',
            # 'record_health': str(row['record_health']) if row['record_health'] else 'Unknown',
            'vote_count': str(row['vote_count']) if row['vote_count'] else 'No Votes Available',
            'rating': str(row['rating']) if row['rating'] else 'No Rating Available',
            'cuisine': str(row['cuisine']) if row['cuisine'] else 'No Cuisine Available',
            'course': str(row['course']) if row['course'] else 'No Course Available',
            'diet': str(row['diet']) if row['diet'] else 'No Diet Information Available',
            'prep_time': str(row['prep_time']) if row['prep_time'] else 'No Prep Time Available',
            'cook_time': str(row['cook_time']) if row['cook_time'] else 'No Cook Time Available',
            'author': str(row['author']) if row['author'] else 'No Author Available',
            'category': str(row['category']) if row['category'] else 'No Category Available'
        }

        # Combine all text fields for the document content
        text = f"{row['description']} {row['ingredients']} {row['instructions']} {row['tags']}"
        doc = Document(page_content=text, metadata=metadata)
        documents.append(doc)
    
    return documents

In [9]:
documents = create_documents(df)

In [10]:
documents[0]

Document(metadata={'recipe_title': 'Roasted Peppers And Mushroom Tortilla Pizza Recipe', 'url': 'https://www.archanaskitchen.com/roasted-peppers-and-mushroom-tortilla-pizza-recipe', 'vote_count': '434', 'rating': '4.9585253456221', 'cuisine': 'Mexican', 'course': 'Dinner', 'diet': 'Vegetarian', 'prep_time': '15 M', 'cook_time': '15 M', 'author': 'Divya Shivaraman ', 'category': 'Pizza Recipes'}, page_content=' is a quicker version pizza to satisfy your cravings. It is a very quick and easy recipe for days that you do not feel like cooking a full fledged meal. With the preference of toppings of your choice this pizza recipe is definitely a winner at any home. The toppings used in this  has some roasted peppers, mushroom with loaded cheese and marinara sauce. Enjoy this easy recipe with your favorite toppings.\xa0 This is a great recipe, if you are looking for an Indian/Fusion Pizza or a Homemade Pizza recipe. Serve  along with  \xa0and   for a weekend night dinner. If you like this reci

In [11]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [12]:
split_documents = split_documents_with_metadata(documents, text_splitter_recursive)

In [13]:
from transformers import AutoTokenizer

def count_tokens(text, model_name="distilbert-base-uncased"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    tokens = tokenizer.encode(text, add_special_tokens=False)
    num_tokens = len(tokens)
    
    return num_tokens

#### This uses in memory qdrant database so we can experiment faster without having to reinitialize the kendra database. We should progress to using a hosted solution like Kendra or self host qdrant on AWS once document chunking and ranking is in a good state.

In [14]:
# This is faster than deleting and rebuilding the kendra db
# but it still takes a few minutes to run
from langchain_community.vectorstores import Qdrant

qdrant_store = Qdrant.from_documents(split_documents,
    embedding_model,
    location=":memory:",
)

In [15]:
qdrant_retriever = qdrant_store.as_retriever()

In [34]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

### Agent function definitions

In [16]:
recipe_db_query_tool = {
  "name": "query_food_recipe_vector_db",
  "description": """
      Queries the vector database containing food recipes to retrieve the most relevant documents. 
      This function allows the model to generate and execute multiple queries as necessary to gather comprehensive context, 
      such as ingredients, preparation steps, and metadata like cuisine and diet type, ensuring accurate and thorough responses to user queries.
      """,
  "input_schema": {
    "type": "object",
    "properties": {
      "queries": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "A query generated by the model to run against the vector database to fetch recipe documents."
        },
        "description": "A list of queries generated by the model to run against the vector database to fetch recipe documents."
      }
    },
    "required": ["queries"]
  }
}


### Init bedrock model, define util to stateless messaging, no fn calling

In [17]:
import json

In [18]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [48]:
# We will need to tune these prompts
# query_bedrock_llm() definition NEEDS TO BE RERUN
# each time when changes are made to this prompt

baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

Before answering, always make at least one call to query_food_recipe_vector_db
to retrieve the relevant context of recipes and ingredients to generate an informed
and high-quality response to the user prompt but NEVER exceed a MAXIMUM of 
3 calls to the query_food_recipe_vector_db function.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [49]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    print("CALLING WITH MESSAGES")
    print(messages)
    print("=======")
    
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )

    response_body = json.loads(response.get('body').read())
    print(f"BEDROCK GOT RESPONSE BODY {response_body})")

    return response_body

### Pipe langchain together

In [262]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [399]:
baseline_user_prompt = """
### Here is a user prompt:
{query}
"""

In [203]:
def process_prompt(query_args):
    prompt_with_query = baseline_user_prompt.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

In [206]:
qdrant_rag_chain = (
    RunnableMap(
        # {"context": qdrant_retriever | format_docs,
         {"query": RunnablePassthrough()}
    )
    | process_prompt
    | query_bedrock_llm
    # | parse_event_stream
)

### Model generates dynamic context queries to vector db

In [29]:
test_query_1 = "I enjoy asian fusion food and I am a vegetarian. Give me one recipe with ingredients and instructions"

In [208]:
qdrant_rag_chain.invoke(test_query_1)

{'ResponseMetadata': {'RequestId': 'aa3d93a7-ead5-457b-87d8-1b8140b86927', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Jul 2024 08:34:56 GMT', 'content-type': 'application/json', 'content-length': '375', 'connection': 'keep-alive', 'x-amzn-requestid': 'aa3d93a7-ead5-457b-87d8-1b8140b86927', 'x-amzn-bedrock-invocation-latency': '3703', 'x-amzn-bedrock-output-token-count': '39', 'x-amzn-bedrock-input-token-count': '549'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x7fa439ab6560>}
{'id': 'msg_bdrk_017xvdoKgaXAYQ9YHoMXiTy7', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'tool_use', 'id': 'toolu_bdrk_01ShrYTuneuhxYy6CeAAwT5X', 'name': 'query_food_recipe_vector_db', 'input': {'queries': ['asian fusion vegetarian recipes']}}], 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 549, 'output_tokens': 39}}


{'id': 'msg_bdrk_017xvdoKgaXAYQ9YHoMXiTy7',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01ShrYTuneuhxYy6CeAAwT5X',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['asian fusion vegetarian recipes']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 549, 'output_tokens': 39}}

In [30]:
test_query_2 = """
I have a peanut allergy but I like thai food. 
I also don't enjoy spicy food much, and want a meal with low carbs. 
Give a recipe with ingredients and instructions
"""

In [210]:
qdrant_rag_chain.invoke(test_query_2)

{'ResponseMetadata': {'RequestId': '5f0f7185-0324-411a-8950-a3e0077e050d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Jul 2024 08:34:59 GMT', 'content-type': 'application/json', 'content-length': '390', 'connection': 'keep-alive', 'x-amzn-requestid': '5f0f7185-0324-411a-8950-a3e0077e050d', 'x-amzn-bedrock-invocation-latency': '2490', 'x-amzn-bedrock-output-token-count': '52', 'x-amzn-bedrock-input-token-count': '573'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x7fa439ab7100>}
{'id': 'msg_bdrk_01XHfJsQF2ivYKKhsvqBLiuh', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'tool_use', 'id': 'toolu_bdrk_01KaXnaTHSYcgn7bvyNWdTN9', 'name': 'query_food_recipe_vector_db', 'input': {'queries': ['thai food', 'peanut free', 'low carb', 'not spicy']}}], 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 573, 'output_tokens': 52}}


{'id': 'msg_bdrk_01XHfJsQF2ivYKKhsvqBLiuh',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01KaXnaTHSYcgn7bvyNWdTN9',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 573, 'output_tokens': 52}}

In [31]:
test_query_3 = """
Suggest a low-carb breakfast recipe that includes eggs and spinach, 
can be prepared in under 20 minutes, 
and is suitable for a keto diet.
"""

In [212]:
qdrant_rag_chain.invoke(test_query_3)

{'ResponseMetadata': {'RequestId': '017cd959-0727-4b43-b0d2-a356f924f166', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Jul 2024 08:35:00 GMT', 'content-type': 'application/json', 'content-length': '417', 'connection': 'keep-alive', 'x-amzn-requestid': '017cd959-0727-4b43-b0d2-a356f924f166', 'x-amzn-bedrock-invocation-latency': '1399', 'x-amzn-bedrock-output-token-count': '59', 'x-amzn-bedrock-input-token-count': '568'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x7fa439ab7d60>}
{'id': 'msg_bdrk_01J3kpLzprvAfwKB1EQyWmvD', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'tool_use', 'id': 'toolu_bdrk_01HKkw3US7DbmUiQbF8ib9uD', 'name': 'query_food_recipe_vector_db', 'input': {'queries': ['low-carb breakfast recipe', 'eggs', 'spinach', 'keto diet', 'under 20 minutes']}}], 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 568, 'output_tokens

{'id': 'msg_bdrk_01J3kpLzprvAfwKB1EQyWmvD',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01HKkw3US7DbmUiQbF8ib9uD',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['low-carb breakfast recipe',
     'eggs',
     'spinach',
     'keto diet',
     'under 20 minutes']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 568, 'output_tokens': 59}}

In [32]:
test_query_4 = """
Suggest a healthy dinner recipe for two people that includes fish, 
is under 500 calories per serving, 
and can be made in less than 40 minutes.
"""

In [214]:
qdrant_rag_chain.invoke(test_query_4)

{'ResponseMetadata': {'RequestId': '1346cce7-5457-4d65-be81-f24b90bbb0c8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Jul 2024 08:35:02 GMT', 'content-type': 'application/json', 'content-length': '439', 'connection': 'keep-alive', 'x-amzn-requestid': '1346cce7-5457-4d65-be81-f24b90bbb0c8', 'x-amzn-bedrock-invocation-latency': '1413', 'x-amzn-bedrock-output-token-count': '58', 'x-amzn-bedrock-input-token-count': '567'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x7fa4402a8310>}
{'id': 'msg_bdrk_01NDSdtdVPyTv1JKecy9wynf', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'tool_use', 'id': 'toolu_bdrk_01QbmRJxKc7FoKavEFdF5QcW', 'name': 'query_food_recipe_vector_db', 'input': {'queries': ['healthy dinner recipe with fish', 'under 500 calories per serving', 'less than 40 minutes to make']}}], 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens':

{'id': 'msg_bdrk_01NDSdtdVPyTv1JKecy9wynf',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01QbmRJxKc7FoKavEFdF5QcW',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['healthy dinner recipe with fish',
     'under 500 calories per serving',
     'less than 40 minutes to make']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 567, 'output_tokens': 58}}

### Implement continuous dialogue and function calling

In [22]:
def generate_message(prompt):
    if type(prompt) != str:
        raise ValueError(f'Tried to call message generate_message with non-string input: {prompt}')
        
    return {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt
            }
        ]
    }

In [23]:
def generate_tool_message(fn_results):
    
    return {
        "role": "user",
        "content": fn_results
    }

In [50]:
import json

# Adds the current prompt as a new message to the chat history
# and calls bedrock with the entire chat history
# Returns the response body, llm's message, and new chat history

'''
Example response body structure:
{
   "id":"msg_bdrk_01C5GGkafK7aL3P5i3rsMr1p",
   "type":"message",
   "role":"assistant",
   "model":"claude-3-sonnet-20240229",
   "content":[
      {
         "type":"tool_use",
         "id":"toolu_bdrk_01CQiYa8BMJfpJC68DuRdwQn",
         "name":"query_food_recipe_vector_db",
         "input":{
            "queries":[
               "healthy fish dinner recipe under 500 calories",
               "fish dinner recipe for two under 40 minutes"
            ]
         }
      }
   ],
   "stop_reason":"tool_use",
   "stop_sequence":"None",
   "usage":{
      "input_tokens":559,
      "output_tokens":55
   }
}
'''
def message_handler(existing_chat_history, prompt, is_tool_message=False):
    # Fn results is an array of tool response objects
    # message structure needs to reflect that
    if is_tool_message:
        user_message = generate_tool_message(prompt)
    else:
        user_message = generate_message(prompt)
    existing_chat_history.append(user_message)

    # Parse the response content
    response_body = query_bedrock_llm(existing_chat_history)
    llm_message = {
        'role': response_body['role'],
        'content': response_body['content']
    }

    # Add the response message to the chat history
    existing_chat_history.append(llm_message)
    
    return [response_body, llm_message, existing_chat_history]

In [51]:
# Executes a list of queries and returns a list of document results
def handle_vector_db_queries(queries, retriever=qdrant_retriever): 
    context_docs = []
    for query in queries:
        query_results = retriever.invoke(query)
        context_docs.extend(query_results)

    return context_docs

In [52]:
# Takes as an argument to LLM message content, returns a list of the fn result objects
def handle_function_calls(tool_call_message_content):
    tool_results = []
    
    for tool_call in tool_call_message_content:
        # Only process messages from the LLM that are function calls
        if tool_call['type'] != 'tool_use':
            continue
        fn_id = tool_call['id']
        fn_name = tool_call['name']
        fn_args = tool_call['input']
        fn_result = {
            "type": "tool_result",
            "tool_use_id": fn_id,
        }   

        if fn_name == 'query_food_recipe_vector_db':
            if 'queries' not in fn_args:
                print(f"ERROR: Tried to call {fn_name} with invalid args {fn_args}, skipping..")
                fn_result['content'] = ""
                fn_result['is_error'] = True
                tool_results.append(fn_result)
                continue
                
            print(f"Calling {fn_name} with args {fn_args}")
            context_docs = handle_vector_db_queries(fn_args['queries'])
            context_str = format_docs(context_docs)
            fn_result['content'] = context_str
            tool_results.append(fn_result)
            
        # TODO: handle web search invocation here
        
        else:
            print(f"ERROR: Attempted call to unknown function {fn_name}")
            fn_result['content'] = ""
            fn_result['is_error'] = True
            tool_results.append(fn_result)

    return tool_results

In [53]:
'''
Example payload structure of response_body:

{'id': 'msg_bdrk_01REesjegNiLteurBoxW7pSt',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'tool_use',
   'id': 'toolu_bdrk_01191W2FuAFTRoDqKKeJSmmn',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 573, 'output_tokens': 52}}


Example payload structure of llm_message['content']:

[{'type': 'tool_use',
   'id': 'toolu_bdrk_01191W2FuAFTRoDqKKeJSmmn',
   'name': 'query_food_recipe_vector_db',
   'input': {'queries': ['thai food',
     'peanut free',
     'low carb',
     'not spicy']}}]
'''

# This function is the entry point to invoke the LLM with support for function calling
# parsing output, calling requested functions, sending output is handled here
def run_chat_loop(prompt):
    response_body, llm_message, chat_history = message_handler(existing_chat_history=[], prompt=prompt)
    
    # The model wants to call tools, call them, provide response, repeat until content is generated
    print("RESPOSNE BODY")
    print(response_body)
    while response_body['stop_reason'] == 'tool_use':
        print("RESPOSNE BODY")
        print(response_body)
        fn_results = handle_function_calls(tool_call_message_content=llm_message['content'])

        # Send function results back to LLM as a new message with the existing chat history
        response_body, llm_message, chat_history = message_handler(
            existing_chat_history=chat_history, 
            prompt=fn_results,
            is_tool_message=True
        )

    # The model is done calling tools
    print(llm_message)

In [54]:
run_chat_loop(test_query_4)

CALLING WITH MESSAGES
[{'role': 'user', 'content': [{'type': 'text', 'text': '\nSuggest a healthy dinner recipe for two people that includes fish, \nis under 500 calories per serving, \nand can be made in less than 40 minutes.\n'}]}]
BEDROCK GOT RESPONSE BODY {'id': 'msg_bdrk_01GRu1kMoYTzTcKAXzLXMTtm', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'tool_use', 'id': 'toolu_bdrk_017NkLdC4xXjWGbjKccQzxfz', 'name': 'query_food_recipe_vector_db', 'input': {'queries': ['healthy fish dinner recipe under 500 calories quick']}}], 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 482, 'output_tokens': 70}})
RESPOSNE BODY
{'id': 'msg_bdrk_01GRu1kMoYTzTcKAXzLXMTtm', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'tool_use', 'id': 'toolu_bdrk_017NkLdC4xXjWGbjKccQzxfz', 'name': 'query_food_recipe_vector_db', 'input': {'queries': ['healthy fish dinner recipe under 500 calor